In [1]:
#!pip install -q ultralytics PyYAML
%cd ~
%cd /kaggle/working/
!git clone https://github.com/apdhar/fork_dcn_yolo
%cd /kaggle/working/fork_dcn_yolo
!pip install -e .

/root
/kaggle/working
Cloning into 'fork_dcn_yolo'...
remote: Enumerating objects: 2221, done.
remote: Counting objects: 100% (296/296), done.
remote: Compressing objects: 100% (290/290), done.
remote: Total 2221 (delta 242), reused 6 (delta 6), pack-reused 1925 (from 2)
Receiving objects: 100% (2221/2221), 2.18 MiB | 13.37 MiB/s, done.
Resolving deltas: 100% (1299/1299), done.
/kaggle/working/fork_dcn_yolo
Obtaining file:///kaggle/working/fork_dcn_yolo
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━

In [2]:
import datetime
import shutil
from pathlib import Path
from collections import Counter

import yaml
import numpy as np
import pandas as pd
from ultralytics import YOLO
from sklearn.model_selection import KFold
import glob, os
from PIL import Image

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
TARGET_IMAGES_PATH = '/kaggle/working/ALL_DATASET/full_dataset/images/'
TARGET_LABELS_PATH = '/kaggle/working/ALL_DATASET/full_dataset/labels/'

source_folder_path = "/kaggle/input/foodds-final/Dataset/{split}/{type}"
splits = ["train", "valid"]
types = ["images/*.jpg", "labels/*.txt"]

os.makedirs(TARGET_IMAGES_PATH, exist_ok=True)
os.makedirs(TARGET_LABELS_PATH, exist_ok=True)

image_paths = list()
label_paths = list()

for split in splits:
    for data_type in types:
        files = glob.glob(source_folder_path.format(split=split, type=data_type))
        for file_ in files:
            if "image" in data_type:
                shutil.copy(file_, TARGET_IMAGES_PATH)
            else:
                shutil.copy(file_, TARGET_LABELS_PATH)

# Store image and label paths for future use
image_paths = glob.glob(TARGET_IMAGES_PATH + "*.jpg")
label_paths = glob.glob(TARGET_LABELS_PATH + "*.txt")

In [4]:
dataset_path = Path('/kaggle/working/ALL_DATASET/full_dataset') # replace with 'path/to/dataset' for your custom data
labels = sorted(dataset_path.rglob("*labels/*.txt")) # all data in 'labels'

yaml_file = '/kaggle/input/foodds-final/Dataset/data.yaml'  # your data YAML with data directories and names dictionary
with open(yaml_file, 'r', encoding="utf8") as y:
    classes = yaml.safe_load(y)['names']
cls_idx = list(range(len(classes)))
print(list(zip(classes, cls_idx)))


[('Beguni', 0), ('Coin', 1), ('Peaju', 2), ('Puri', 3), ('Singara', 4), ('Somusa', 5)]


In [5]:
indx = [l.stem for l in labels] # uses base filename as ID (no extension)
labels_df = pd.DataFrame([], columns=cls_idx, index=indx)

In [6]:
for label in labels:
    lbl_counter = Counter()

    with open(label,'r') as lf:
        lines = lf.readlines()

    for l in lines:
        # classes for YOLO label uses integer at first position of each line
        lbl_counter[int(l.split(' ')[0])] += 1

    labels_df.loc[label.stem] = lbl_counter

labels_df = labels_df.fillna(0.0) # replace `nan` values with `0.0`
labels_df

/tmp/ipykernel_19/3131205890.py:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  labels_df = labels_df.fillna(0.0) # replace `nan` values with `0.0`


,0,1,2,3,4,5
00000001_resized_png.rf.96da3a5cd0e190e00aa4f24987f72848,0.0,0.0,7.0,0.0,0.0,0.0
00000002_resized_png.rf.17018bf1b1ef711c31e89e349d3b89e1,0.0,0.0,16.0,0.0,0.0,0.0
00000002_resized_png.rf.71c93ebfd030ffad185e53dcade6dbfe,0.0,0.0,16.0,0.0,0.0,0.0
00000006_resized_png.rf.3eed21752d5275f1dcafe7f612f90023,0.0,0.0,17.0,0.0,0.0,0.0
00000006_resized_png.rf.48bdac5b83c70bbeeff465851964c3e2,0.0,0.0,17.0,0.0,0.0,0.0
...,...,...,...,...,...,...
samos-214-_jpg.rf.8a60dd9b70a16bc815f62f4e6fba15af,0.0,0.0,0.0,0.0,0.0,2.0
samos-214-_jpg.rf.a0f47c396dd83f83023bb0bd835e6019,0.0,0.0,0.0,0.0,0.0,2.0
samos-219-_jpg.rf.437520264886f45e41d9b8beb8c0aaff,0.0,0.0,0.0,0.0,4.0,0.0
samos-223-_jpg.rf.27eae3b4d9dc9b79e83a786eba296ddf,0.0,0.0,0.0,0.0,0.0,5.0


In [7]:
ksplit = 5
kf = KFold(n_splits=ksplit, shuffle=True, random_state=20)   # setting random_state for repeatable results

kfolds = list(kf.split(labels_df))

In [8]:
folds = [f'split_{n}' for n in range(1, ksplit + 1)]
fold_lbl_distrb = pd.DataFrame(index=folds, columns=cls_idx)

for n, (train_indices, val_indices) in enumerate(kfolds, start=1):
    train_totals = labels_df.iloc[train_indices].sum()
    val_totals = labels_df.iloc[val_indices].sum()

    # To avoid division by zero, we add a small value (1E-7) to the denominator
    ratio = val_totals / (train_totals + 1E-7)
    fold_lbl_distrb.loc[f'split_{n}'] = ratio

In [9]:
fold_lbl_distrb


,0,1,2,3,4,5
split_1,0.282667,0.268872,0.25446,0.23221,0.23491,0.23842
split_2,0.222363,0.240464,0.234383,0.287671,0.23491,0.231707
split_3,0.236504,0.272109,0.286175,0.234522,0.259567,0.245205
split_4,0.254237,0.240464,0.231839,0.202925,0.263773,0.288448
split_5,0.255875,0.229252,0.244664,0.29783,0.257475,0.247769


In [10]:
kfold_base_path = Path('Main-DATA/kfold')
shutil.rmtree(kfold_base_path) if kfold_base_path.is_dir() else None # Remove existing folder
os.makedirs(str(kfold_base_path)) # Create nww folder
yaml_paths = list()
train_txt_paths = list()
val_txt_paths = list()
for i, (train_idx, val_idx) in enumerate(kfolds):
    # Get image paths for train-val split
    train_paths = [image_paths[j] for j in train_idx]
    val_paths = [image_paths[j] for j in val_idx]
    # Create text files to store image paths
    train_txt = kfold_base_path / f"train_{i}.txt"
    val_txt =  kfold_base_path / f"val_{i}.txt"

    # Write images paths for training and validation in split i
    with open(str(train_txt), 'w') as f:
        f.writelines(s + '\n' for s in train_paths)
    with open(str(val_txt), 'w') as f:
        f.writelines(s + '\n' for s in val_paths)

    train_txt_paths.append(str(train_txt))
    val_txt_paths.append(str(val_txt))

    # Create yaml file
    yaml_path = kfold_base_path / f'data_{i}.yaml'
    with open(yaml_path, 'w') as ds_y:
        yaml.safe_dump({
            'train': str(train_txt.name),
            'val': str(val_txt.name),
            'names': classes
        }, ds_y)
    yaml_paths.append(str(yaml_path))
print("Yaml Paths")
print(yaml_paths)

Yaml Paths
['Main-DATA/kfold/data_0.yaml', 'Main-DATA/kfold/data_1.yaml', 'Main-DATA/kfold/data_2.yaml', 'Main-DATA/kfold/data_3.yaml', 'Main-DATA/kfold/data_4.yaml']


In [11]:
print("data_0.yaml File: \n")
!cat Main-DATA/kfold/data_0.yaml

print("\ntrain_0.txt first two lines: \n")
!cat Main-DATA/kfold/train_0.txt | head -n 2

data_0.yaml File: 

names:
- Beguni
- Coin
- Peaju
- Puri
- Singara
- Somusa
train: train_0.txt
val: val_0.txt

train_0.txt first two lines: 

/kaggle/working/ALL_DATASET/full_dataset/images/IMG_20241118_135719_jpg.rf.708330914dcb29186f5e5d9b1c08feac.jpg
/kaggle/working/ALL_DATASET/full_dataset/images/IMG_20241127_143359_jpg.rf.ba8d03245166f14502a746c1f1a33126.jpg
cat: write error: Broken pipe


In [12]:
from IPython.display import clear_output
import time
os.environ['WANDB_DISABLED'] = 'true'

epochs = 125
optimizer="AdamW"
lr0=0.001
pretrained=True
imgsz=640
batch = 32
project = 'kfold_demo'


results = list()

for i in range(ksplit):
    model = YOLO("/kaggle/working/fork_dcn_yolo/ultralytics/cfg/models/v8/yolov8n-seg-CD.yaml")
    dataset_yaml = "/kaggle/working/fork_dcn_yolo/" + yaml_paths[i]
    print(f"Training for fold={i} using {dataset_yaml}")
    model.train(data=dataset_yaml, batch=batch, project=project, epochs=epochs, verbose=False, workers=28)
    result = model.metrics # Metrics on validation set
    results.append(result) # save output metrics for further analysis
    clear_output()

In [13]:
metric_values = dict()

for result in results:
    for metric, metric_val in result.results_dict.items():
        if metric not in metric_values:
            metric_values[metric] = []
        metric_values[metric].append(metric_val)

metric_df = pd.DataFrame.from_dict(metric_values)
visualize_metric = ['mean', 'std', 'min', 'max']
metric_df.describe().loc[visualize_metric]

,metrics/precision(B),metrics/recall(B),metrics/mAP50(B),metrics/mAP50-95(B),metrics/precision(M),metrics/recall(M),metrics/mAP50(M),metrics/mAP50-95(M),fitness
mean,0.947096,0.936343,0.973725,0.872369,0.937252,0.918988,0.956580,0.810548,1.707657
std,0.007899,0.007460,0.005289,0.010066,0.012083,0.011326,0.008387,0.015515,0.023955
min,0.938603,0.929896,0.967609,0.865469,0.923531,0.904823,0.946750,0.795508,1.686886
max,0.960025,0.944768,0.980436,0.889301,0.955113,0.932334,0.969453,0.834785,1.746666


In [14]:
import pandas as pd

# Assuming `results` is a list of objects, each with a `results_dict` attribute (a dictionary of metrics)
metric_values = dict()

# Collect metrics from all results
for result in results:
    for metric, metric_val in result.results_dict.items():
        if metric not in metric_values:
            metric_values[metric] = []
        metric_values[metric].append(metric_val)

# Create DataFrame where each column is a metric and each row is a fold
metric_df = pd.DataFrame.from_dict(metric_values)

# Show values for each fold
print("Metric values per fold:")
print(metric_df)

# Show selected statistics
visualize_metric = ['mean', 'std', 'min', 'max']
print("\nDescriptive statistics:")
print(metric_df.describe().loc[visualize_metric])


Metric values per fold:
   metrics/precision(B)  metrics/recall(B)  metrics/mAP50(B)  \
0              0.960025           0.944109          0.980436   
1              0.938603           0.929896          0.967609   
2              0.946635           0.944768          0.977758   
3              0.946142           0.930392          0.970299   
4              0.944075           0.932551          0.972524   

   metrics/mAP50-95(B)  metrics/precision(M)  metrics/recall(M)  \
0             0.889301              0.955113           0.932334   
1             0.866103              0.923531           0.912486   
2             0.866975              0.931062           0.928361   
3             0.865469              0.934014           0.916938   
4             0.873999              0.942538           0.904823   

   metrics/mAP50(M)  metrics/mAP50-95(M)   fitness  
0          0.969453             0.834785  1.746666  
1          0.946750             0.795508  1.686886  
2          0.958486          